# 5 强化学习

假设玩飞行棋, 丢骰子决定往前多少步, 且只能动一个棋子**没有其他选择**. 这是含有概率的问题. 

假设所有状态 (位置) 为 $s_1,s_2,\dotsc$, 第 $t+1$ 时间的状态是基于 $t$ 时间状态的概率分布
$p(s^{(t+1)}|s^{(t)})$.

## 马尔科夫过程 

### 马尔科夫过程

若如上所述, 第 $t+1$ 时间的状态概率分布只与 $t$ 时间状态有关, 而与之前的状态无关, 则称为马尔科夫过程 (Markov process)
$$p(s^{(t+1)}|s^{(t)})=p(s^{(t+1)}|s^{(t)},\dotsc,s^{(1)})$$


### 马尔科夫奖励模型

假设每个状态 $s$ 有奖励 $R(s)$. 那么从 $s^{(t)}$ 开始, 未来的奖励之和的期望是

$$V(s)=\mathbb E(R(s^{(t)}) + R(s^{(t+1)}) + R(s^{(t+2)})+\dotsc|s^{(t)})=\sum_{n=t}^\infty\mathbb E(R(s^{(n)})|s^{(t)}).$$

为了保证收敛性, 有时会引入参数衰减因子 $\gamma\in (0,1)$
$$V_\gamma (s^{(t)}) =\mathbb E(R(s^{(t)}) + \gamma R(s^{(t+1)}) + \gamma^2 R(s^{(t+2)})+\dotsc|s^{(t)})= \sum_{n=t}^\infty\gamma^{n-t}\mathbb E(R(s^{(n)})|s^{(t)}).$$

称为马尔科夫奖励模型 (Markov reward process, MRP).

<br>

显然上式可以递推, 因为 
$$\mathbb E(\gamma R(s^{(t+1)}) + \gamma^2 R(s^{(t+2)})+\dotsc|s^{(t)})
=\gamma \sum \mathbb E(R(s^{(t+1)}) + \gamma R(s^{(t+2)})+\dotsc |s^{(t+1)})p(s^{(t+1)}|s^{(t)})$$

即
$$V_\gamma (s^{(t)}) =R(s^{(t)})+\gamma  \sum_{s^{(t+1)}}V_\gamma (s^{(t+1)})p(s^{(t+1)}|s^{(t)}).$$


虽然上式看起来是递推, 实际上可能是线性方程组 (因为未来的状态也可能回到先前的状态), 可用迭代法求解.


### 马尔科夫决策过程

假设每一步不仅有概率, 我们还可以选择, 假设选择记为 $a$, 第 $t+1$ 步的状态服从关于 $t$ 时的状态与选择确定的概率分布:
$p(s^{(t+1)}|s^{(t)},a^{(t)})$. 目标是选择一系列 $a$ 使得总奖励最大. 这是马尔科夫决策过程 (Markov decision process).

即 

$$V_\gamma (s^{(t)},a^{(t)}) =R(s^{(t)})+\gamma  \sum_{s^{(t+1)}}V_\gamma (s^{(t+1)})p(s^{(t+1)}|s^{(t)},a^{(t)}).$$

若选取关于 $a$ 的最大值, 则


$$V^*_\gamma (s^{(t)}) ={\argmax}_a\left\{R(s^{(t)})+\gamma  \sum_{s^{(t+1)}}V^*_\gamma (s^{(t+1)})p(s^{(t+1)}|s^{(t)},a)\right\}.$$


虽然上式看起来是递推, 实际上可能是线性方程组 (因为未来的状态也可能回到先前的状态), 可用迭代法求解, 称为贝尔曼 (Bellman) 迭代.

In [60]:
# 规则: 如下地图, S起点, H为正确终点到达得+1分, B为错误终点到达得-1分, W为墙
# 每次可以选择上下左右四个方向移动, 
# 移动有 80% 成功, 20% 则会往选择的垂直方向移动 (例如选择往上有10%向左和10%向右)
# 若移动时碰到墙则留在原地
# 请尽量快地达到正确的终点(H)
world = """\
WWWWWW
W   HW
W W BW
WS   W
WWWWWW"""
world = world.split('\n')
w, h = len(world), len(world[0])
score = [[0. for i in range(h)] for j in range(w)]
for i in range(w):
    for j in range(h):
        if world[i][j] == 'H': score[i][j] = 1.
        if world[i][j] == 'B': score[i][j] = -1.

def ShowScore(scores):
    f = lambda x: ' %.3f'%(x) if x >= 0 else '%.3f'%(x)
    print('\n'.join((
        '  '.join(f(s) for s in score[1:-1])) for score in scores[1:-1]))
ShowScore(score)

 0.000   0.000   0.000   1.000
 0.000   0.000   0.000  -1.000
 0.000   0.000   0.000   0.000


In [57]:
def BellmanIteration(world, score, gamma = .8):
    w, h = len(world), len(world[0])
    p = 0.8
    new_score = [[s for s in line] for line in score]
    for i in range(w):
        for j in range(h):
            if world[i][j] == ' ' or world[i][j] == 'S':
                max_s = -2147483648
                for dx, dy in ((1,0),(0,1),(-1,0),(0,-1)):
                    # for each direction (action)
                    s = 0
                    
                    # has probabiliy p to go straight (when encounter walls, it remains still)
                    s += p * score[i+dx][j+dy] if world[i+dx][j+dy] != 'W' else p * score[i][j]
                    
                    # has probability (1-p) to go aside
                    dx, dy = dy, dx
                    s += (1-p)/2 * score[i+dx][j+dy] if world[i+dx][j+dy] != 'W' else (1-p)/2 * score[i][j]
                    
                    dx, dy = -dx, -dy
                    s += (1-p)/2 * score[i+dx][j+dy] if world[i+dx][j+dy] != 'W' else (1-p)/2 * score[i][j]
                    
                    max_s = max(max_s, s)
                new_score[i][j] = max_s * gamma # note that reward = 0 except at termination
    return new_score

In [63]:
new_score = [[s for s in line] for line in score] # deepcopy
for iters in range(5):
    new_score = BellmanIteration(world, new_score, gamma = .9)
    ShowScore(new_score)
    print('')

 0.000   0.000   0.720   1.000
 0.000   0.000   0.000  -1.000
 0.000   0.000   0.000   0.000

 0.000   0.518   0.785   1.000
 0.000   0.000   0.428  -1.000
 0.000   0.000   0.000   0.000

 0.373   0.658   0.829   1.000
 0.000   0.000   0.514  -1.000
 0.000   0.000   0.308   0.000

 0.508   0.716   0.841   1.000
 0.269   0.000   0.553  -1.000
 0.000   0.222   0.370   0.132

 0.585   0.734   0.845   1.000
 0.414   0.000   0.565  -1.000
 0.213   0.306   0.430   0.188



实际上不一定要每轮(使用深拷贝)基于上一轮的所有 V 值进行更新, 直接利用新的 V 值更新亦可.